In [ ]:
#This project is created by Hashmitha Shantharam for IMarche Inc.


In [1]:
#importing beautifulsoup(Python library for extracting structured data from a website) for web scarping
!pip3 install beautifulsoup4

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [3]:
#to scrape details from amazon bookstore
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [4]:
#save the result in csv
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [6]:
#read the csv file for interpretations
df.shape

(100, 5)

In [5]:
df = pd.read_csv("C:/Users/Prasad Rajaraman/Desktop/Data Mining/Python/amazon_products.csv")

In [7]:
df.head(61)

,Book Name,Author,Rating,Customers_Rated,Price
0,Karma: A Yogi's Guide to Crafting Your Destiny,Sadhguru,4.8 out of 5 stars,27,₹ 264.00
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"21,603",₹ 399.00
2,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"13,498",₹ 295.00
3,My First Book of Patterns Pencil Control: Patt...,Wonder House Books,4.4 out of 5 stars,"4,864",₹ 89.00
4,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6 out of 5 stars,"8,224",₹ 625.00
...,...,...,...,...,...
56,NCERT Textbooks in English for class 12 - Flam...,NCERT,4.5 out of 5 stars,300,₹ 100.00
57,"Indian Economy for Civil Services, Universitie...",Ramesh Singh,4.3 out of 5 stars,"1,451",₹ 475.00
58,The Blue Umbrella,Ruskin Bond,4.5 out of 5 stars,"4,634",₹ 95.00
59,India & World Map ( Both Political & Physical ...,JCB GROUP,4.8 out of 5 stars,560,₹ 250.00


In [8]:
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])

In [9]:
df['Rating'] = pd.to_numeric(df['Rating'])

In [10]:
df["Price"] = df["Price"].str.replace('₹', '')

In [11]:
df["Price"] = df["Price"].str.replace(',', '')

In [12]:
df['Price'] = df['Price'].apply(lambda x: x.split('.')[0])

In [13]:
df['Price'] = df['Price'].astype(int)

In [14]:
df["Customers_Rated"] = df["Customers_Rated"].str.replace(',', '')

In [15]:
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='ignore')

In [16]:
df.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,Karma: A Yogi's Guide to Crafting Your Destiny,Sadhguru,4.8,27,264
1,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,21603,399
2,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,13498,295
3,My First Book of Patterns Pencil Control: Patt...,Wonder House Books,4.4,4864,89
4,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,8224,625


In [17]:
df.dtypes

Book Name           object
Author              object
Rating             float64
Customers_Rated      int64
Price                int32
dtype: object

In [18]:
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)

In [19]:
count_nan = len(df) - df.count()

In [20]:
count_nan

Book Name          0
Author             1
Rating             0
Customers_Rated    0
Price              0
dtype: int64

In [21]:
df = df.dropna()

In [22]:
data = df.sort_values(["Price"], axis=0, ascending=False)[:15]

In [23]:
data

,Book Name,Author,Rating,Customers_Rated,Price
82,New Simplified Physics : A Reference Book for ...,S.L. Arora,4.3,1395,1050
89,My First Complete Learning Library: Boxset of ...,Wonder House Books,4.6,4115,799
15,Atomic Habits: The life-changing million copy ...,James Clear,4.6,18016,646
4,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,8224,625
25,Objective NCERT at your FINGERTIPS for NEET-AI...,MTG Editorial Board,4.6,1334,562
51,A Modern Approach to Verbal & Non-Verbal Reaso...,R.S. Aggarwal,4.4,4864,549
23,Mathematics for Class 10 by R D Sharma (Examin...,R.D. Sharma,4.6,1377,540
46,The Intelligent Investor (English) Paperback –...,Benjamin Graham,4.5,22496,539
95,Indian Art and Culture for Civil Services and ...,Nitin Singhania,4.6,2413,507
97,All set to Read-A Phonic Reader-Level 1- PHONI...,Om Books Editorial Team,4.6,327,487


In [24]:
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Legend
output_notebook()

Loading BokehJS ...

In [30]:
from bokeh.transform import factor_cmap
from bokeh.models import Legend
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.palettes import d3
#colors has a list of colors which can be used in plots
colors = itertools.cycle(palette)

palette = d3['Category20'][20]

In [31]:
#most customer rated author
index_cmap = factor_cmap('Author', palette=palette,
                         factors=data["Author"])

In [34]:
p = figure(plot_width=700, plot_height=700, title = "Top Authors: Rating vs. Customers Rated")
p.scatter('Rating','Customers_Rated',source=data,fill_alpha=0.6, fill_color=index_cmap,size=20,legend='Author')
p.xaxis.axis_label = 'RATING'
p.yaxis.axis_label = 'CUSTOMERS RATED'
p.legend.location = 'top_left'

In [35]:
show(p)